In [ ]:
import numpy as np
from functools import reduce
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

#read data and make pd dataframe
insurance_data = pd.read_csv("./Insurance_Data.csv")
insurance_data.head()
data_frame = pd.DataFrame(insurance_data)


# Removing non categorical features and status for now 
cols = [1,2,3,10,11]
data_frame.drop(data_frame.columns[cols], axis=1, inplace=True)

#count the amount of classes for each categorical feature
class_count = {}
for feature in data_frame:
    classes = data_frame[feature].value_counts()
    class_count[feature] = classes

print(class_count.keys())


#encode y
y = data_frame["Conclusion"]
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_cat = to_categorical(y_encoded)


#encode x
X = data_frame.drop(columns=["Conclusion"])
X = pd.get_dummies(X)

#split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, train_size=0.8, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)


classes_df = pd.DataFrame(classes)
classes_num = classes_df.sum()
print(classes_num.iloc[0])

model = keras.Sequential()
model.add(layers.Input(shape=(X_train.shape[1],) ))
model.add(layers.Dense(512, activation="relu"))
model.add(layers.Dense(y_cat.shape[1], activation="softmax"))


model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])


early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(
 X_train, y_train,
 validation_data=(X_val, y_val),
 batch_size=256,
 epochs=500,
 callbacks=[early_stopping]
)

model.summary()
loss, acc = model.evaluate(X_test, y_test)
print(loss, acc)

dict_keys(['Company', 'Coverage', 'SubCoverage', 'Reason', 'SubReason', 'Disposition', 'Conclusion'])
21173
Epoch 1/500
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.4321 - loss: 2.8850 - val_accuracy: 0.5386 - val_loss: 1.7018
Epoch 2/500
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.5691 - loss: 1.5369 - val_accuracy: 0.5919 - val_loss: 1.3676
Epoch 3/500
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6267 - loss: 1.2644 - val_accuracy: 0.6157 - val_loss: 1.2562
Epoch 4/500
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6616 - loss: 1.1093 - val_accuracy: 0.6374 - val_loss: 1.1899
Epoch 5/500
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6715 - loss: 1.0548 - val_accuracy: 0.6435 - val_loss: 1.1525
Epoch 6/500
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6855 - loss: 1.0031 - val_accuracy: 0.6451 - val_loss: 1.1283
Epoch 7/500
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6946 - loss: 0.9488 - val_accuracy: 0.6458 - val_loss: 1.1

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │       571,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 53)             │        27,189 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,797,281 (6.86 MB)

 Trainable params: 599,093 (2.29 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,198,188 (4.57 MB)

240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6498 - loss: 1.0875
1.090462327003479 0.6476352214813232
